**Задание**

Пакет SURPRISE:

* используйте данные MovieLens 1M,
* можно использовать любые модели из пакета,
* получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:
в домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K.
Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

Установка библиотеки  surprise и ее модулей, загрузка исходных датасетов:

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811677 sha256=22dcc8699a938fce30e427e83e68d0a807fab793d4dc2f3f0c8328eb810a05a1
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans, KNNBasic, SVD, KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

import pandas as pd

In [ ]:
!wget 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip' -O MovieLens.zip

--2023-08-06 15:01:35--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 955.28K  3.03MB/s    in 0.3s    

2023-08-06 15:01:36 (3.03 MB/s) - ‘MovieLens.zip’ saved [978202/978202]



In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

Для выполнения задания воспользуемся датасетами movies и ratings.
Количество уникальных значений в них:

In [ ]:
movies.nunique()

movieId    9742
title      9737
genres      951
dtype: int64

In [ ]:
ratings.nunique()

userId         610
movieId       9724
rating          10
timestamp    85043
dtype: int64

Т.к. в датасете movies 5 фильмов имеют по 2 movieID (всего movieId 9742, а title 9737 шт.), найдем их и удалим дубликаты.

In [ ]:
duplicateRows = movies[movies.duplicated('title')]
duplicateRows

,movieId,title,genres
5601,26958,Emma (1996),Romance
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [ ]:
movies_new = movies.drop_duplicates(['title'])

In [ ]:
movies_new.nunique()

movieId    9737
title      9737
genres      951
dtype: int64

Т.к. фильмов больше, чем пользователей, воспользуемся User-based подходом.

In [ ]:
# Соберем датасет для выполнения задания:
movies_with_ratings = movies_new.merge(ratings, on='movieId', how='outer')
movies_with_ratings.drop(columns=['timestamp', 'genres'], inplace=True)
movies_with_ratings[:3]

,movieId,title,userId,rating
0,1,Toy Story (1995),1.0,4.0
1,1,Toy Story (1995),5.0,4.0
2,1,Toy Story (1995),7.0,4.5


In [ ]:
movies_with_ratings.shape

(100854, 4)

In [ ]:
# Найдем и удалим строки со значениями NaN:
movies_with_ratings.isnull().sum()

movieId     0
title       6
userId     18
rating     18
dtype: int64

In [ ]:
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100830 entries, 0 to 100847
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100830 non-null  int64  
 1   title    100830 non-null  object 
 2   userId   100830 non-null  float64
 3   rating   100830 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.8+ MB


Данные для работы алгоритмов будем брать из датафрейма.

In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0)) # Минимальные и максимальные значения оценок - как в лекции (0,5 и 5 соответственно)
data = Dataset.load_from_df(dataset, reader)

Проверим работу нескольких алгоритмов с параметрами по умолчанию, выберем лучший.

In [ ]:
# Список с алгоритмами:
algo_list = [
    KNNWithMeans(),
    KNNBasic(),
    SVD(),
    KNNBaseline()
    ]

In [ ]:
for algo in algo_list:
  cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
  print('___________________________________________________________\n')

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9024  0.8979  0.8945  0.8933  0.8923  0.8961  0.0037  
Fit time          0.17    0.18    0.22    0.21    0.23    0.20    0.02    
Test time         1.51    1.97    1.69    1.69    2.05    1.78    0.20    
___________________________________________________________

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matri

Самое низкое среднее значение RMSE дает алгоритм SVD(). Для снижения RMSE до 0,87 и ниже, настроим параметры алгоритма вручную.

In [ ]:
algo_SVD = SVD(n_factors=150, n_epochs=15, biased=True,
               init_mean=0, init_std_dev=0.01, lr_all=0.01,
               reg_all=0.02, random_state=9)

In [ ]:
cross_validate(algo_SVD, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8654  0.8584  0.8685  0.8544  0.8639  0.8621  0.0050  
Fit time          2.31    2.08    2.07    1.66    2.28    2.08    0.23    
Test time         0.49    0.16    0.43    0.19    0.40    0.33    0.13    


{'test_rmse': array([0.8654172 , 0.85841358, 0.86845152, 0.8544188 , 0.8639445 ]),
 'fit_time': (2.308910369873047,
  2.0797410011291504,
  2.0731844902038574,
  1.6633188724517822,
  2.2750256061553955),
 'test_time': (0.49105381965637207,
  0.15589594841003418,
  0.42530107498168945,
  0.18816113471984863,
  0.3988311290740967)}

Цель достигнута: среднее значение RMSE снизилось с 0,8742 до 0,8621.